In [2]:
1. Baza danych(#baza)

SyntaxError: invalid syntax (<ipython-input-2-9a27ad2a81d9>, line 1)

In [3]:
import btk
from ezc3d import c3d
import pandas as pd
import numpy as np
import glob
import h5py
import math
from pyomeca import Markers

In [4]:
def cropp_c3dfile(eventsFrame, filename, destiny):
    reader = btk.btkAcquisitionFileReader()
    reader.SetFilename(filename)
    reader.Update()
    acq = reader.GetOutput()
 
    writer = btk.btkAcquisitionFileWriter()
    
    for i in range(0, len(eventsFrame)):
        clone = acq.Clone();
        clone.ResizeFrameNumberFromEnd(acq.GetLastFrame() - eventsFrame[i][0] + 1)
        clone.ResizeFrameNumber(eventsFrame[i][1] - eventsFrame[i][0] + 1)
        clone.SetFirstFrame(eventsFrame[i][0])
        clone.ClearEvents()
        for e in btk.Iterate(acq.GetEvents()):
            if ((e.GetFrame() > clone.GetFirstFrame()) and (e.GetFrame() < clone.GetLastFrame())):
                clone.AppendEvent(e)
        clone.SetFirstFrame(1)
        writer.SetInput(clone)
        writer.SetFilename(destiny + '\\' + (filename.split('\\')[-1]).split('.')[0] + '.c3d')
        writer.Update()

In [5]:
path = f'medical_dataset_AK'

In [6]:
filelist =[]
for file in glob.glob(f'{path}\\**\\*.c3d',recursive = True):
    filelist.append(file)

In [7]:
def read_labels(sample):
    e_label = pd.DataFrame(sample['parameters']['EVENT']['LABELS']['value'])
    e_contexts = pd.DataFrame(sample['parameters']['EVENT']['CONTEXTS']['value'])

    times = sample['parameters']['EVENT']['TIMES']['value']
    e_frames = pd.DataFrame(times[1] * 100).astype(int)

    event = pd.concat([e_label, e_contexts, e_frames], axis=1)
    event.columns = ['label', 'context', 'frames']
    event = event.set_index('frames')
    event = event.sort_index(axis=0)

    return event

In [8]:
sample = c3d('medical_dataset_AK\degeneration\degeneration_01.c3d')
times = read_labels(sample).T
times

frames,146,171,225,244,298,309,365,374,434,444,501,511,571,580,636,646,709,725,802
label,Foot Strike,Foot Off,Foot Strike,Foot Off,Foot Strike,Foot Off,Foot Strike,Foot Off,Foot Strike,Foot Off,Foot Strike,Foot Off,Foot Strike,Foot Off,Foot Strike,Foot Off,Foot Strike,Foot Off,Foot Off
context,Right,Left,Left,Right,Right,Left,Left,Right,Right,Left,Left,Right,Right,Left,Left,Right,Right,Left,Right


In [9]:
def get_one_step_time(sample, label, context):
    df = read_labels(sample)
    temp = [frame for frame in df.index if df['label'][frame] == label and df['context'][frame] == context]

    return temp[0], temp[1]

In [10]:
for file in filelist:
    sample = c3d(file)
    start, stop = get_one_step_time(sample, 'Foot Strike', 'Right')
    eventsFrame = [[int(start), int(stop)]]
    cropp_c3dfile(eventsFrame, file, 'cut')

In [11]:
marker_list = ['LANK', 'RANK', 'LKNE', 'RKNE', 'LFIN', 'RFIN', 'LSHO', 'RSHO', 'LASI', 'RFHD','RWRA','RHEE','LHEE','CentreOfMass']

In [12]:
path = f'cut'
filelist =[]
for file in glob.glob(f'{path}\\*.c3d',recursive = True):
    filelist.append(file)
filelist

['cut\\degeneration_01.c3d',
 'cut\\degeneration_02.c3d',
 'cut\\degeneration_03.c3d',
 'cut\\degeneration_04.c3d',
 'cut\\degeneration_05.c3d',
 'cut\\degeneration_06.c3d',
 'cut\\degeneration_07.c3d',
 'cut\\degeneration_08.c3d',
 'cut\\degeneration_09.c3d',
 'cut\\degeneration_10.c3d',
 'cut\\degeneration_11.c3d',
 'cut\\degeneration_12.c3d',
 'cut\\degeneration_13.c3d',
 'cut\\degeneration_14.c3d',
 'cut\\degeneration_15.c3d',
 'cut\\healthy_01.c3d',
 'cut\\healthy_02.c3d',
 'cut\\healthy_03.c3d',
 'cut\\healthy_04.c3d',
 'cut\\healthy_05.c3d',
 'cut\\healthy_06.c3d',
 'cut\\healthy_07.c3d',
 'cut\\healthy_08.c3d',
 'cut\\healthy_09.c3d',
 'cut\\healthy_10.c3d',
 'cut\\healthy_11.c3d',
 'cut\\healthy_12.c3d',
 'cut\\healthy_13.c3d',
 'cut\\healthy_14.c3d',
 'cut\\healthy_15.c3d']

In [13]:
avg = 127

In [14]:
from pyomeca import Markers

In [15]:
def data_markers(data_path, marker_list):
    data_markers = Markers.from_c3d(data_path, usecols=[marker_list[0]])
    data_markers = data_markers.meca.time_normalize(n_frames=avg)
    data_markers = data_markers.meca.to_wide_dataframe()
    for i in range(len(marker_list)-1):
        
        tmp_markers = Markers.from_c3d(data_path, usecols=[marker_list[i+1]])
        tmp_markers = tmp_markers.meca.time_normalize(n_frames=avg)  
        tmp_markers = tmp_markers.meca.to_wide_dataframe()
        data_markers = data_markers.join(tmp_markers)
 
    cols = [c for c in data_markers.columns if c.lower()[:4] != 'ones']
    markers_dataframe = data_markers[cols]
    
    return markers_dataframe

In [16]:
dm = data_markers(filelist[0], marker_list)
dat = np.array(dm['x_LASI'])
dat

array([81.748909  , 81.61628517, 81.39627693, 81.07543546, 80.64097171,
       80.07818192, 79.36220551, 78.50192854, 77.49314638, 76.33459691,
       75.0215947 , 73.55195836, 71.94869486, 70.22211481, 68.384385  ,
       66.44375865, 64.41728029, 62.32864319, 60.19479316, 58.03317327,
       55.86234429, 53.70357005, 51.57489098, 49.49339282, 47.47512708,
       45.54477904, 43.70984868, 41.9764655 , 40.350349  , 38.83395658,
       37.44065185, 36.15207212, 34.95998637, 33.85575104, 32.83061921,
       31.88209364, 30.99127769, 30.15083122, 29.35503672, 28.60113598,
       27.88941023, 27.21436125, 26.5755736 , 25.97284468, 25.40784236,
       24.88096237, 24.3878938 , 23.9272341 , 23.49742354, 23.09910329,
       22.73035952, 22.38825716, 22.07217601, 21.78190928, 21.52056558,
       21.28819263, 21.08616532, 20.91835485, 20.79006649, 20.71479664,
       20.69733465, 20.74622878, 20.87191082, 21.08533859, 21.41374085,
       21.85478259, 22.41870308, 23.1146642 , 23.95359169, 24.95

In [17]:
def load_group(filenames):
    
    loaded = []
    
    for file in filenames:
        try:
            result = data_markers(file, marker_list)
            loaded.append(result)
        except:
            print(file)

    return loaded

In [18]:
group = load_group(filelist)

cut\healthy_01.c3d
cut\healthy_10.c3d
cut\healthy_14.c3d


### Brakuje 4 plików, dlaczego czas sie nie zgrywa?

Baza danych <a name="baza"></a>

In [19]:
x = np.stack(group)
x.shape

(27, 127, 42)

In [20]:
print('(samples, timesteps, features) -> ', x.shape)

(samples, timesteps, features) ->  (27, 127, 42)


Kolejność, 'LANK' dla x, y, z, 'RANK' dla x, y, z ...... itd.

In [21]:
def create_h5_basic(path, x):
    f = h5py.File(path, mode='w')
    f.create_dataset("x", data=x)
    f.close()

In [22]:
def create_h5(path, x, y):
    f = h5py.File(path, mode='w')
    f.create_dataset("x", data=x)
    f.create_dataset("y", data=y)
    f.close()

In [23]:
create_h5_basic("ak_basic.h5", x)

In [24]:
f=h5py.File(r'ak_basic.h5', 'r')
print(list(f.keys()))
x = f['x']
print('(samples, timesteps, features) -> ', x.shape)

['x']
(samples, timesteps, features) ->  (27, 127, 42)


# JRD

In [25]:
import numpy as np

def jrd_method(markers):
    
    #arrays with data
    x_lank = np.array(markers['x_LANK'])
    y_lank = np.array(markers['y_LANK'])
    z_lank = np.array(markers['z_LANK'])
    
    x_rank = np.array(markers['x_RANK'])
    y_rank = np.array(markers['y_RANK'])
    z_rank = np.array(markers['z_RANK'])

    #auxiliary tables
    arr_distance_jrd = []
    arr_left = []
    arr_right = []
    time = []
    i = 0
    x_2 = 0
    y_2 = 0
    z_2 = 0

    while i < avg:
    
        x_2 = (x_lank[i] - x_rank[i]) ** 2
        y_2 = (y_lank[i] - y_rank[i]) ** 2
        z_2 = (z_lank[i] - z_rank[i]) ** 2
        distance = math.sqrt(x_2+y_2+z_2)
        
        time.append(i)
        arr_left.append(z_lank[i])
        arr_right.append(z_rank[i])
        arr_distance_jrd.append(distance)
    
        i += 1
        
    return arr_distance_jrd

# JRA

In [26]:
import numpy as np

def jra_method(markers):
    
    #arrays with data
    x_lasi = np.array(markers['x_LASI'])
    y_lasi = np.array(markers['y_LASI'])
    z_lasi = np.array(markers['z_LASI'])
    
    x_lkne = np.array(markers['x_LKNE'])
    y_lkne = np.array(markers['y_LKNE'])
    z_lkne = np.array(markers['z_LKNE'])
    
    x_lank = np.array(markers['x_LANK'])
    y_lank = np.array(markers['y_LANK'])
    z_lank = np.array(markers['z_LANK'])
    
    #auxiliary tables
    first_dist = []
    second_dist = []
    distance = []
    angle = []
    time = []
    
    i = 0
    xl = yl = zl = 0
    xl_1 = yl_1 = zl_1 = 0

    while i < avg:
        
        xl = (x_lasi[i] - x_lkne[i])
        yl = (y_lasi[i] - y_lkne[i])
        zl = (z_lasi[i] - z_lkne[i])
        
        xl_1 = (x_lkne[i] - x_lank[i])
        yl_1 = (y_lkne[i] - y_lank[i])
        zl_1 = (z_lkne[i] - z_lank[i])
        
        time.append(i)
    
        first_dist.append(math.sqrt((xl ** 2) + (yl ** 2) + (zl ** 2)))
        second_dist.append(math.sqrt((xl_1 ** 2) + (yl_1 ** 2) + (zl_1 ** 2)))                   
        distance.append(xl * xl_1 + yl * yl_1 + zl * zl_1)
    
        angle.append(math.acos(distance[i] / (first_dist[i] * second_dist[i])))
        
        i += 1
        
    return angle


# HDF

In [27]:
import numpy as np

from scipy.stats import skew 

def hdf_method(markers):

    #arrays with data
    x_lank = np.array(markers['x_LANK'])
    x_rank = np.array(markers['x_RANK'])
    x_lkne = np.array(markers['x_LKNE'])
    x_rkne = np.array(markers['x_RKNE'])
    x_lfin = np.array(markers['x_LFIN'])
    x_rfin = np.array(markers['x_RFIN'])
    x_lsho = np.array(markers['x_LSHO'])
    x_rsho = np.array(markers['x_RSHO'])
    
    #auxiliaty tables
    #hd1 ankles
    hd1 = []
    hd1_x = 0
    #hd2 knees
    hd2 = []
    hd2_x = 0
    #hd3 wrists
    hd3 = []
    hd3_x = 0
    #hd4 shoulders
    hd4 = []
    hd4_x = 0
    
    #variables
    meanH = []
    stdH = []
    skewH = []
    HDF = [] 
    hdf = []
    time = []
    i = 0
    
    while i < avg:
    
        hd1_x = (x_lank[i] - x_rank[i]) ** 2
        hd1.append(math.sqrt(hd1_x))
        
        hd2_x = (x_lkne[i] - x_rkne[i]) ** 2
        hd2.append(math.sqrt(hd2_x))
        
        hd3_x = (x_lfin[i] - x_rfin[i]) ** 2
        hd3.append(math.sqrt(hd3_x))
        
        hd4_x = (x_lsho[i] - x_rsho[i]) ** 2
        hd4.append(math.sqrt(hd4_x))
        
        time.append(i)
        hdf = [hd1[i], hd2[i], hd3[i], hd4[i]]
        
        meanH.append(np.mean(hdf))
        stdH.append(np.std(hdf))
        skewH.append(skew(hdf))        
        HDF.append([meanH, stdH, skewH])
        
        i += 1
        
    return HDF[0][0], HDF[0][1], HDF[0][2]

# VDF

In [28]:
import numpy as np

def vdf_method(markers):
    
    #arrays with data
    y_rfhd = np.array(markers['y_RFHD'])
    y_rwra = np.array(markers['y_RWRA'])
    y_rsho = np.array(markers['y_RSHO'])
    y_rank = np.array(markers['y_RANK'])
    y_lank = np.array(markers['y_LANK'])
    y_rhee = np.array(markers['y_RHEE'])
    y_lhee = np.array(markers['y_LHEE'])
    y_centreOfMass = np.array(markers['y_CentreOfMass'])
    
    #auxiliary tables
    #vd1 HEIGHT
    vd1 = []
    vd1_y = 0
    #vd2 WRIST RIGHT
    vd2 = []
    vd2_y = 0
    #vd3 SHOULDER RIGHT
    vd3 = []
    vd3_y = 0
    #vd4 ANKLE RIGHT
    vd4 = []
    vd4_y = 0
    #vd5 ANKLE LEFT
    vd5 = []
    vd5_y = 0
    #vd6 DIST. FEET LEFT AND RIGHT
    vd6 = []
    vd6_y = 0
    
    #variables
    meanV = []
    stdV = []
    VDF = [] 
    vdf = []
    time = []
    average = []
    i = 0
    
    while i < avg:
    
        vd1_y = np.array(y_rfhd[i])
        vd1.append(vd1_y)
        
        vd2_y = np.array(y_rwra[i])
        vd2.append(vd2_y)
        
        vd3_y = np.array(y_rsho[i])
        vd3.append(vd3_y)
        
        vd4_y = np.array(y_rank[i])
        vd4.append(vd4_y)
        
        vd5_y = np.array(y_lank[i])
        vd5.append(vd5_y)
        
        vd6_y = np.array((0.5 * (y_rhee[i] - y_lhee[i]) * y_centreOfMass[i]))
        vd6.append(vd6_y)
        
        time.append(i)
        
        average = (vd1_y + vd2_y + vd3_y + vd4_y + vd5_y + vd6_y)
        vdf = [vd1[i], vd2[i], vd3[i], vd4[i], vd5[i], vd6[i]]
        
        meanV.append(np.mean(average))
        stdV.append(np.std(vdf))
        VDF.append([meanV, stdV])
        
        i += 1
        
    return VDF[0][0], VDF[0][1]

## New approach

In [33]:
def data_features(data_path):
    marker_list = ['LANK', 'RANK', 'LKNE', 'RKNE', 'LFIN', 'RFIN', 'LSHO', 'RSHO', 'LASI', 'RFHD','RWRA','RHEE','LHEE','CentreOfMass']
    markers = data_markers(data_path, marker_list)
    jrd = jrd_method(markers)
    jra = jra_method(markers)
    meanH, stdH, skewH = hdf_method(markers)
    meanV, stdV = vdf_method(markers)
    
    result = [jrd, jra, meanH, stdH, skewH, meanV, stdV]
    
    return pd.DataFrame(result).T

In [34]:
data = data_features(filelist[0])
data

,0,1,2,3,4,5,6
0,465.300408,0.115936,325.351617,133.957345,0.711506,-423119.334219,162248.767781
1,466.316187,0.114198,325.363847,133.866037,0.714171,-422846.731320,162137.849766
2,466.845857,0.112334,325.374542,133.796605,0.716902,-422008.223288,161815.992006
3,467.038160,0.110398,325.387928,133.750138,0.719596,-420775.503278,161347.104380
4,467.033516,0.108464,325.410560,133.725369,0.722190,-419305.867428,160789.736369
...,...,...,...,...,...,...,...
122,408.301987,0.140139,333.049385,150.984911,0.744349,-208970.138279,80526.915828
123,415.853276,0.139225,332.657330,151.240664,0.740297,-212655.084197,81885.144201
124,421.655912,0.138264,332.285305,151.510600,0.736226,-215162.295449,82805.082572
125,425.899233,0.137337,331.954995,151.782401,0.732462,-216620.878523,83334.743518


In [32]:
import config as cfg

In [33]:
final_db = []
final_label = []

for file in filelist:
    try:
        db_features = data_features(file)
        final_db.append(db_features)
        final_label.append(file[4:-4].replace("_", ""))
    except:
        print(file)

cut\healthy_01.c3d
cut\healthy_10.c3d
cut\healthy_14.c3d


Błąd został naprawiony.

In [34]:
dbl = pd.DataFrame({'File' : pd.Series(final_label)})
dbl.shape, dbl

((27, 1),
               File
 0   degeneration01
 1   degeneration02
 2   degeneration03
 3   degeneration04
 4   degeneration05
 5   degeneration06
 6   degeneration07
 7   degeneration08
 8   degeneration09
 9   degeneration10
 10  degeneration11
 11  degeneration12
 12  degeneration13
 13  degeneration14
 14  degeneration15
 15       healthy02
 16       healthy03
 17       healthy04
 18       healthy05
 19       healthy06
 20       healthy07
 21       healthy08
 22       healthy09
 23       healthy11
 24       healthy12
 25       healthy13
 26       healthy15)

In [35]:
# create_h5('features_db_test6.h5', final_db, dbl)

In [36]:
create_h5_basic('features_database_jrd_jra_one_cycle_right.h5', final_db)

Lista nazw plików ---->

In [36]:
final_label

['degeneration01',
 'degeneration02',
 'degeneration03',
 'degeneration04',
 'degeneration05',
 'degeneration06',
 'degeneration07',
 'degeneration08',
 'degeneration09',
 'degeneration10',
 'degeneration11',
 'degeneration12',
 'degeneration13',
 'degeneration14',
 'degeneration15',
 'healthy02',
 'healthy03',
 'healthy04',
 'healthy05',
 'healthy06',
 'healthy07',
 'healthy09',
 'healthy11',
 'healthy13',
 'healthy14',
 'healthy15']